In [1]:
import re, os, sys
from functools import reduce
from itertools import combinations as comb, permutations as perm, combinations_with_replacement as combr
from operator import itemgetter
from pprint import pprint
from math import *
from collections import defaultdict
import networkx as nx
import numpy as np

def dprint(d):
    pprint(dict(d))
def lmap(*args, **kwargs):
    return list(map(*args, **kwargs))
def lmapi(*args, **kwargs):
    return list(map(int, *args, **kwargs))
def enum(*args, **kwargs):
    return enumerate(*args, **kwargs)

In [82]:
input_fname = "../inputs/day23-demo.in"

In [93]:
input_fname = "../inputs/day23.in"

In [94]:
def reread():
    lines = []
    with open(input_fname) as ifile:
        def r(): return ifile.readline().strip()
        def rs(): return ifile.readline().strip().split()
        def ri(): return int(ifile.readline().strip())
        def ris(): return list(map(int, ifile.readline().strip().split()))

        # decide between two possible methods of reading input
        
        rows = []
        # method 1
        for line in ifile:
            rows.append([1 if c == "#" else 0 for c in line.strip()])
        return np.array(rows)

ar = reread()
# ar

In [34]:
def pad_as_needed(ar):
    if sum(ar[0]):
        ar = np.pad(ar, ((1,0), (0,0)))
    if sum(ar[-1]):
        ar = np.pad(ar, ((0,1), (0,0)))
    if sum(ar[:,0]):
        ar = np.pad(ar, ((0,0), (1,0)))
    if sum(ar[:,-1]):
        ar = np.pad(ar, ((0,0), (0,1)))
    return ar``

In [35]:
dirs = np.array([[i,j] for i in (-1,0,1) for j in (-1,0,1) if i or j])
NW, N, NE, W, E, SW, S, SE = dirs

## Part 1

In [36]:
def check(ar, x, y, dis):
    for dy, dx in dis:
        if ar[y+dy, x+dx]:
            # print("found at", dy, dx)
            return False
    return True

In [37]:
checkdirs = [(N, NE, NW), (S, SE, SW), (W, NW, SW), (E, NE, SE)]
from itertools import cycle

In [106]:
def do_round(ar):
    elves = list(zip(*ar.nonzero()))
    proposals = dict()
    # first half, consider to move
    for y, x in elves:
        # check if any surrounding elves
        if np.sum(ar[y-1:y+2,x-1:x+2]) == 1:
            # print(y, x, "no elves around")
            continue
        ny = nx = None
        for i in range(4):
            if check(ar, x, y, checkdirs[(r+i)%4]):
                ny, nx = checkdirs[(r+i)%4][0] + [y, x]
                # print(y, x, "free to move", ny, nx)
                break
        if ny is None and nx is None:
            # print(y, x, "elves all around")
            continue
        if (ny, nx) not in proposals:
            # print(y, x, "found free", ny, nx)
            proposals[(ny, nx)] = y, x
        else:
            # print(y, x, "clashed", ny, nx)
            proposals[(ny, nx)] = None
    
    # second half, actually move
    rr = ar.copy()
    for (ny, nx), src in proposals.items():
        if src is None: continue
        rr[ny, nx] = 1
        sy, sx = src
        assert ar[sy, sx] == 1, "no elf found at src"
        rr[sy, sx] = 0
    return rr

In [107]:
rr = pad_as_needed(ar)
for r in range(10):
    rr = do_round(rr)
    rr = pad_as_needed(rr)
# for row in rr: print("".join(".#"[e] for e in row))

In [101]:
def trim_array(ar):
    if not sum(ar[0]):
        ar = ar[1:]
    if not sum(ar[-1]):
        ar = ar[:-1]
    if not sum(ar[:,0]):
        ar = ar[:,1:]
    if not sum(ar[:,-1]):
        ar = ar[:,:-1]
    return ar

In [102]:
fr = trim_array(rr)
print(fr.size - np.sum(fr))

4052


## Part 2

In [103]:
def do_round(ar, r):
    elves = list(zip(*ar.nonzero()))
    proposals = dict()
    # first half, consider to move
    for y, x in elves:
        # check if any surrounding elves
        if np.sum(ar[y-1:y+2,x-1:x+2]) == 1:
            # print(y, x, "no elves around")
            continue
        ny = nx = None
        for i in range(4):
            if check(ar, x, y, checkdirs[(r+i)%4]):
                ny, nx = checkdirs[(r+i)%4][0] + [y, x]
                # print(y, x, "free to move", ny, nx)
                break
        if ny is None and nx is None:
            # print(y, x, "elves all around")
            continue
        if (ny, nx) not in proposals:
            # print(y, x, "found free", ny, nx)
            proposals[(ny, nx)] = y, x
        else:
            # print(y, x, "clashed", ny, nx)
            proposals[(ny, nx)] = None
    
    # second half, actually move
    rr = ar.copy()
    moved = False
    for (ny, nx), src in proposals.items():
        moved = True
        if src is None: continue
        rr[ny, nx] = 1
        sy, sx = src
        assert ar[sy, sx] == 1, "no elf found at src"
        rr[sy, sx] = 0
    if moved:
        return rr
    else:
        return None

In [104]:
from tqdm import trange
rr = pad_as_needed(ar)
for r in trange(10000):
    nr = do_round(rr, r)
    if nr is None:
        print("no move round", r+1)
        break
    else:
        rr = pad_as_needed(nr)
# for row in rr: print("".join(".#"[e] for e in row))

 10%|▉         | 977/10000 [00:46<07:13, 20.81it/s]

no move round 978
